In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as st 
from tqdm import tqdm 
import os
## Change directory to your working directory
os.chdir('D:\Shashi Katteri\Desktop\data sets to work on\Amazon Employee Access\Dataset_10')

In [3]:
df=pd.read_csv('Amazon_employee_access.csv')
df.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,target
0,39353,85475,117961,118300,123472,117905,117906,290919,117908,1
1,17183,1540,117961,118343,123125,118536,118536,308574,118539,1
2,36724,14457,118219,118220,117884,117879,267952,19721,117880,1
3,36135,5396,117961,118343,119993,118321,240983,290919,118322,1
4,42680,5905,117929,117930,119569,119323,123932,19793,119325,1


In [4]:
df1=df.astype(str)
df1['target']=df1['target'].astype(int)

In [5]:
#df1['ROLE_ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_TITLE','ROLE_FAMILY'],axis=1)


In [6]:
df1['target']=df1['target'].map({1:0,0:1})

In [7]:
df1['ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_ROLLUP_1','ROLE_ROLLUP_2'],axis=1)

In [8]:
df1.head()

,RESOURCE,MGR_ID,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE,target,ROLLUP_12
0,39353,85475,123472,117906,117908,0,117961118300
1,17183,1540,123125,118536,118539,0,117961118343
2,36724,14457,117884,267952,117880,0,118219118220
3,36135,5396,119993,240983,118322,0,117961118343
4,42680,5905,119569,123932,119325,0,117929117930


# Combinations with brute force 


In [9]:
from itertools import combinations

cols=list(df1.columns)
cols.remove('target')
d=df1.copy()

for j in combinations(cols,2):
    d[j[0]+'_'+j[1]]=df1[j[0]]+df1[j[1]]
    
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32769 entries, 0 to 32768
Data columns (total 22 columns):
RESOURCE                          32769 non-null object
MGR_ID                            32769 non-null object
ROLE_DEPTNAME                     32769 non-null object
ROLE_FAMILY_DESC                  32769 non-null object
ROLE_CODE                         32769 non-null object
target                            32769 non-null int64
ROLLUP_12                         32769 non-null object
RESOURCE_MGR_ID                   32769 non-null object
RESOURCE_ROLE_DEPTNAME            32769 non-null object
RESOURCE_ROLE_FAMILY_DESC         32769 non-null object
RESOURCE_ROLE_CODE                32769 non-null object
RESOURCE_ROLLUP_12                32769 non-null object
MGR_ID_ROLE_DEPTNAME              32769 non-null object
MGR_ID_ROLE_FAMILY_DESC           32769 non-null object
MGR_ID_ROLE_CODE                  32769 non-null object
MGR_ID_ROLLUP_12                  32769 non-null obj

In [10]:
from lightgbm import LGBMClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split,StratifiedKFold,KFold
from catboost import CatBoostClassifier

In [11]:
def aucroc(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    
    return auc

In [12]:
import warnings
warnings.filterwarnings('ignore')
def kfold_AS_target(train,test,columns,target,weight,folds=10):
    
    columns=list(columns)
    kf=KFold()
    t=train
    v=test
    gm=t[target].mean()
    for col in columns:
        
        for i,j in kf.split(t):
            gb=t.loc[t.index[i]].groupby(col)[target].agg(['count','mean'])
            count=gb['count']
            mean=gb['mean']
            gb['smooth']=(count*mean + weight*gm)/(count+weight)
            t.loc[t.index[j],col+'_enc']=t.loc[t.index[j],col].map(gb['smooth'])
            
        t[col+'_enc']=t[col+'_enc'].fillna(gm)
        
        gb=t.groupby(col)[target].agg(['count','mean'])
        count=gb['count']
        mean=gb['mean']
        gb['smooth']=(count*mean + weight*gm)/(count+weight)
        
        v[col+'_enc']=v[col].map(gb['smooth'])
        v[col+'_enc']=v[col+'_enc'].fillna(gm)
        
    t=t.drop(columns,axis=1)
    v=v.drop(columns,axis=1)

    return t,v
        

In [50]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

def cross_val(algo,d,columns,target):
    sf=StratifiedKFold(n_splits=10)
    model=algo
    fold=0
    fold_mod=[]
    x=df.drop(target,axis=1)
    y=df[target]
    for i,j in sf.split(x,y):
        
        sm=SMOTE(sampling_strategy=0.1)
        fold=fold+1

        train=d.iloc[i]
        test=d.iloc[j]
       
        
        c=columns+[target]
        t,v=kfold_AS_target(train[c],test[c],columns,target,100,folds=10)
        

        xtr=t.drop(target,axis=1)
        xts=v.drop(target,axis=1)
        ytr=t[target]
        yts=v[target]
        x,y=sm.fit_resample(xtr,ytr)
        
        mod=model.fit(x,y)

        auc=aucroc(mod,xts,yts)
        fold_mod.append(auc)
    return fold_mod


In [51]:
rem=['MGR_ID_ROLE_DEPTNAME', 'MGR_ID', 'ROLE_DEPTNAME_ROLLUP_12', 'MGR_ID_ROLLUP_12', 'ROLLUP_12', 'ROLE_CODE_ROLLUP_12',
     'RESOURCE_ROLE_FAMILY_DESC']
d1=d.drop(rem,axis=1)

In [52]:
d1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32769 entries, 0 to 32768
Data columns (total 15 columns):
RESOURCE                          32769 non-null object
ROLE_DEPTNAME                     32769 non-null object
ROLE_FAMILY_DESC                  32769 non-null object
ROLE_CODE                         32769 non-null object
target                            32769 non-null int64
RESOURCE_MGR_ID                   32769 non-null object
RESOURCE_ROLE_DEPTNAME            32769 non-null object
RESOURCE_ROLE_CODE                32769 non-null object
RESOURCE_ROLLUP_12                32769 non-null object
MGR_ID_ROLE_FAMILY_DESC           32769 non-null object
MGR_ID_ROLE_CODE                  32769 non-null object
ROLE_DEPTNAME_ROLE_FAMILY_DESC    32769 non-null object
ROLE_DEPTNAME_ROLE_CODE           32769 non-null object
ROLE_FAMILY_DESC_ROLE_CODE        32769 non-null object
ROLE_FAMILY_DESC_ROLLUP_12        32769 non-null object
dtypes: int64(1), object(14)
memory usage: 3.8+ MB


In [53]:
cols=list(d1.columns)
cols.remove('target')
target='target'
params={'learning_rate':0.01,'depth':6,'verbose':False,'eval_metric':'AUC',
        'loss_function':'CrossEntropy'}
cb=CatBoostClassifier()
auc=cross_val(cb,d1,cols,target)
auc_avg=np.array(auc).mean()

Learning rate set to 0.044372
0:	learn: 0.6287667	total: 14.3ms	remaining: 14.3s
1:	learn: 0.5749636	total: 34.2ms	remaining: 17.1s
2:	learn: 0.5269622	total: 59.3ms	remaining: 19.7s
3:	learn: 0.4871809	total: 97.8ms	remaining: 24.4s
4:	learn: 0.4511986	total: 123ms	remaining: 24.5s
5:	learn: 0.4219468	total: 139ms	remaining: 23.1s
6:	learn: 0.3938697	total: 164ms	remaining: 23.3s
7:	learn: 0.3696155	total: 187ms	remaining: 23.2s
8:	learn: 0.3487506	total: 208ms	remaining: 22.9s
9:	learn: 0.3304905	total: 240ms	remaining: 23.8s
10:	learn: 0.3147209	total: 269ms	remaining: 24.2s
11:	learn: 0.3004880	total: 290ms	remaining: 23.9s
12:	learn: 0.2879915	total: 316ms	remaining: 24s
13:	learn: 0.2770367	total: 346ms	remaining: 24.4s
14:	learn: 0.2677652	total: 378ms	remaining: 24.8s
15:	learn: 0.2583922	total: 397ms	remaining: 24.4s
16:	learn: 0.2511698	total: 416ms	remaining: 24s
17:	learn: 0.2443596	total: 442ms	remaining: 24.1s
18:	learn: 0.2383097	total: 472ms	remaining: 24.4s
19:	learn: 

160:	learn: 0.1535085	total: 3.46s	remaining: 18s
161:	learn: 0.1533445	total: 3.48s	remaining: 18s
162:	learn: 0.1532204	total: 3.5s	remaining: 18s
163:	learn: 0.1531276	total: 3.52s	remaining: 17.9s
164:	learn: 0.1530177	total: 3.53s	remaining: 17.9s
165:	learn: 0.1529010	total: 3.54s	remaining: 17.8s
166:	learn: 0.1528104	total: 3.55s	remaining: 17.7s
167:	learn: 0.1526147	total: 3.58s	remaining: 17.7s
168:	learn: 0.1524909	total: 3.59s	remaining: 17.7s
169:	learn: 0.1523352	total: 3.63s	remaining: 17.7s
170:	learn: 0.1521632	total: 3.65s	remaining: 17.7s
171:	learn: 0.1520699	total: 3.68s	remaining: 17.7s
172:	learn: 0.1516980	total: 3.69s	remaining: 17.6s
173:	learn: 0.1515983	total: 3.71s	remaining: 17.6s
174:	learn: 0.1514060	total: 3.73s	remaining: 17.6s
175:	learn: 0.1513113	total: 3.75s	remaining: 17.6s
176:	learn: 0.1511840	total: 3.76s	remaining: 17.5s
177:	learn: 0.1511207	total: 3.78s	remaining: 17.4s
178:	learn: 0.1510206	total: 3.79s	remaining: 17.4s
179:	learn: 0.15090

319:	learn: 0.1358294	total: 5.86s	remaining: 12.5s
320:	learn: 0.1357665	total: 5.88s	remaining: 12.4s
321:	learn: 0.1357190	total: 5.89s	remaining: 12.4s
322:	learn: 0.1356423	total: 5.9s	remaining: 12.4s
323:	learn: 0.1356077	total: 5.91s	remaining: 12.3s
324:	learn: 0.1355216	total: 5.92s	remaining: 12.3s
325:	learn: 0.1353448	total: 5.94s	remaining: 12.3s
326:	learn: 0.1351825	total: 5.95s	remaining: 12.2s
327:	learn: 0.1351263	total: 5.97s	remaining: 12.2s
328:	learn: 0.1350563	total: 5.99s	remaining: 12.2s
329:	learn: 0.1349672	total: 6s	remaining: 12.2s
330:	learn: 0.1348707	total: 6.02s	remaining: 12.2s
331:	learn: 0.1347639	total: 6.03s	remaining: 12.1s
332:	learn: 0.1346206	total: 6.05s	remaining: 12.1s
333:	learn: 0.1345545	total: 6.06s	remaining: 12.1s
334:	learn: 0.1344912	total: 6.07s	remaining: 12.1s
335:	learn: 0.1344069	total: 6.1s	remaining: 12.1s
336:	learn: 0.1343260	total: 6.11s	remaining: 12s
337:	learn: 0.1342421	total: 6.13s	remaining: 12s
338:	learn: 0.1341830

478:	learn: 0.1237031	total: 8.42s	remaining: 9.15s
479:	learn: 0.1236192	total: 8.43s	remaining: 9.13s
480:	learn: 0.1235128	total: 8.45s	remaining: 9.12s
481:	learn: 0.1234525	total: 8.46s	remaining: 9.1s
482:	learn: 0.1233622	total: 8.48s	remaining: 9.07s
483:	learn: 0.1232722	total: 8.5s	remaining: 9.06s
484:	learn: 0.1232072	total: 8.51s	remaining: 9.04s
485:	learn: 0.1231708	total: 8.52s	remaining: 9.02s
486:	learn: 0.1231423	total: 8.54s	remaining: 8.99s
487:	learn: 0.1231091	total: 8.55s	remaining: 8.97s
488:	learn: 0.1230606	total: 8.57s	remaining: 8.95s
489:	learn: 0.1230128	total: 8.58s	remaining: 8.93s
490:	learn: 0.1229347	total: 8.6s	remaining: 8.92s
491:	learn: 0.1229104	total: 8.62s	remaining: 8.9s
492:	learn: 0.1228836	total: 8.65s	remaining: 8.89s
493:	learn: 0.1228296	total: 8.66s	remaining: 8.87s
494:	learn: 0.1227661	total: 8.69s	remaining: 8.86s
495:	learn: 0.1226827	total: 8.7s	remaining: 8.84s
496:	learn: 0.1225744	total: 8.72s	remaining: 8.82s
497:	learn: 0.122

637:	learn: 0.1143723	total: 10.8s	remaining: 6.14s
638:	learn: 0.1143269	total: 10.8s	remaining: 6.13s
639:	learn: 0.1142531	total: 10.9s	remaining: 6.11s
640:	learn: 0.1142373	total: 10.9s	remaining: 6.09s
641:	learn: 0.1141995	total: 10.9s	remaining: 6.07s
642:	learn: 0.1141731	total: 10.9s	remaining: 6.05s
643:	learn: 0.1141027	total: 10.9s	remaining: 6.03s
644:	learn: 0.1140669	total: 10.9s	remaining: 6.01s
645:	learn: 0.1139667	total: 10.9s	remaining: 6s
646:	learn: 0.1139194	total: 11s	remaining: 5.98s
647:	learn: 0.1138719	total: 11s	remaining: 5.96s
648:	learn: 0.1138340	total: 11s	remaining: 5.94s
649:	learn: 0.1138142	total: 11s	remaining: 5.92s
650:	learn: 0.1137866	total: 11s	remaining: 5.91s
651:	learn: 0.1137440	total: 11s	remaining: 5.89s
652:	learn: 0.1137085	total: 11s	remaining: 5.87s
653:	learn: 0.1136573	total: 11.1s	remaining: 5.85s
654:	learn: 0.1136092	total: 11.1s	remaining: 5.83s
655:	learn: 0.1135706	total: 11.1s	remaining: 5.81s
656:	learn: 0.1134974	total: 

...

236:	learn: 0.1422316	total: 4.35s	remaining: 14s
237:	learn: 0.1421131	total: 4.36s	remaining: 14s
238:	learn: 0.1419203	total: 4.38s	remaining: 13.9s
239:	learn: 0.1418504	total: 4.4s	remaining: 13.9s
240:	learn: 0.1417513	total: 4.42s	remaining: 13.9s
241:	learn: 0.1416400	total: 4.43s	remaining: 13.9s
242:	learn: 0.1414641	total: 4.45s	remaining: 13.9s
243:	learn: 0.1413697	total: 4.47s	remaining: 13.9s
244:	learn: 0.1412234	total: 4.49s	remaining: 13.8s
245:	learn: 0.1411164	total: 4.51s	remaining: 13.8s
246:	learn: 0.1410111	total: 4.52s	remaining: 13.8s
247:	learn: 0.1408767	total: 4.54s	remaining: 13.8s
248:	learn: 0.1407814	total: 4.56s	remaining: 13.7s
249:	learn: 0.1406490	total: 4.58s	remaining: 13.7s
250:	learn: 0.1405096	total: 4.59s	remaining: 13.7s
251:	learn: 0.1403825	total: 4.61s	remaining: 13.7s
252:	learn: 0.1402857	total: 4.63s	remaining: 13.7s
253:	learn: 0.1402021	total: 4.65s	remaining: 13.7s
254:	learn: 0.1401421	total: 4.66s	remaining: 13.6s
255:	learn: 0.140

395:	learn: 0.1285181	total: 7.15s	remaining: 10.9s
396:	learn: 0.1284345	total: 7.17s	remaining: 10.9s
397:	learn: 0.1283684	total: 7.19s	remaining: 10.9s
398:	learn: 0.1282303	total: 7.2s	remaining: 10.9s
399:	learn: 0.1281660	total: 7.24s	remaining: 10.9s
400:	learn: 0.1280595	total: 7.25s	remaining: 10.8s
401:	learn: 0.1279824	total: 7.27s	remaining: 10.8s
402:	learn: 0.1279629	total: 7.28s	remaining: 10.8s
403:	learn: 0.1278594	total: 7.3s	remaining: 10.8s
404:	learn: 0.1278327	total: 7.31s	remaining: 10.7s
405:	learn: 0.1277449	total: 7.33s	remaining: 10.7s
406:	learn: 0.1276284	total: 7.35s	remaining: 10.7s
407:	learn: 0.1274040	total: 7.37s	remaining: 10.7s
408:	learn: 0.1273527	total: 7.39s	remaining: 10.7s
409:	learn: 0.1273107	total: 7.4s	remaining: 10.7s
410:	learn: 0.1272736	total: 7.42s	remaining: 10.6s
411:	learn: 0.1271596	total: 7.44s	remaining: 10.6s
412:	learn: 0.1270784	total: 7.46s	remaining: 10.6s
413:	learn: 0.1270110	total: 7.48s	remaining: 10.6s
414:	learn: 0.1

554:	learn: 0.1185079	total: 9.85s	remaining: 7.9s
555:	learn: 0.1184900	total: 9.87s	remaining: 7.88s
556:	learn: 0.1184731	total: 9.88s	remaining: 7.86s
557:	learn: 0.1184563	total: 9.9s	remaining: 7.84s
558:	learn: 0.1183854	total: 9.92s	remaining: 7.83s
559:	learn: 0.1182896	total: 9.94s	remaining: 7.81s
560:	learn: 0.1182707	total: 9.96s	remaining: 7.79s
561:	learn: 0.1182467	total: 9.98s	remaining: 7.78s
562:	learn: 0.1181752	total: 10s	remaining: 7.76s
563:	learn: 0.1181595	total: 10s	remaining: 7.74s
564:	learn: 0.1180794	total: 10s	remaining: 7.72s
565:	learn: 0.1180685	total: 10.1s	remaining: 7.71s
566:	learn: 0.1180391	total: 10.1s	remaining: 7.69s
567:	learn: 0.1179863	total: 10.1s	remaining: 7.67s
568:	learn: 0.1179063	total: 10.1s	remaining: 7.65s
569:	learn: 0.1178565	total: 10.1s	remaining: 7.64s
570:	learn: 0.1178461	total: 10.1s	remaining: 7.62s
571:	learn: 0.1177765	total: 10.2s	remaining: 7.6s
572:	learn: 0.1177267	total: 10.2s	remaining: 7.58s
573:	learn: 0.1175947

713:	learn: 0.1102030	total: 12.8s	remaining: 5.11s
714:	learn: 0.1101350	total: 12.8s	remaining: 5.09s
715:	learn: 0.1100910	total: 12.8s	remaining: 5.08s
716:	learn: 0.1100589	total: 12.8s	remaining: 5.06s
717:	learn: 0.1100001	total: 12.8s	remaining: 5.04s
718:	learn: 0.1099178	total: 12.8s	remaining: 5.02s
719:	learn: 0.1098626	total: 12.9s	remaining: 5.01s
720:	learn: 0.1098561	total: 12.9s	remaining: 4.99s
721:	learn: 0.1098093	total: 12.9s	remaining: 4.97s
722:	learn: 0.1097792	total: 12.9s	remaining: 4.95s
723:	learn: 0.1097558	total: 13s	remaining: 4.94s
724:	learn: 0.1096926	total: 13s	remaining: 4.92s
725:	learn: 0.1096000	total: 13s	remaining: 4.91s
726:	learn: 0.1095591	total: 13s	remaining: 4.89s
727:	learn: 0.1094931	total: 13s	remaining: 4.87s
728:	learn: 0.1094528	total: 13s	remaining: 4.85s
729:	learn: 0.1094004	total: 13.1s	remaining: 4.83s
730:	learn: 0.1093550	total: 13.1s	remaining: 4.81s
731:	learn: 0.1092810	total: 13.1s	remaining: 4.8s
732:	learn: 0.1092388	tot

872:	learn: 0.1033936	total: 15.5s	remaining: 2.26s
873:	learn: 0.1033647	total: 15.6s	remaining: 2.24s
874:	learn: 0.1033159	total: 15.6s	remaining: 2.22s
875:	learn: 0.1032483	total: 15.6s	remaining: 2.21s
876:	learn: 0.1032304	total: 15.6s	remaining: 2.19s
877:	learn: 0.1031788	total: 15.6s	remaining: 2.17s
878:	learn: 0.1031592	total: 15.7s	remaining: 2.15s
879:	learn: 0.1031392	total: 15.7s	remaining: 2.14s
880:	learn: 0.1031118	total: 15.7s	remaining: 2.12s
881:	learn: 0.1030764	total: 15.7s	remaining: 2.1s
882:	learn: 0.1030612	total: 15.7s	remaining: 2.08s
883:	learn: 0.1030163	total: 15.8s	remaining: 2.07s
884:	learn: 0.1029904	total: 15.8s	remaining: 2.05s
885:	learn: 0.1029616	total: 15.8s	remaining: 2.03s
886:	learn: 0.1029200	total: 15.8s	remaining: 2.01s
887:	learn: 0.1028846	total: 15.8s	remaining: 2s
888:	learn: 0.1028558	total: 15.8s	remaining: 1.98s
889:	learn: 0.1028068	total: 15.9s	remaining: 1.96s
890:	learn: 0.1027391	total: 15.9s	remaining: 1.94s
891:	learn: 0.10

In [54]:
auc_avg

0.8895784109982818

In [55]:
 auc

[0.8776437892370537,
 0.8888554774241303,
 0.9217209690893902,
 0.8983086969123488,
 0.8784111639643326,
 0.9010877533970982,
 0.876613301962389,
 0.8815840621963071,
 0.8906262254275144,
 0.8809326703722558]